## read file

In [1]:
lines = []
with open('evaluation_all.txt') as f:
    lines = f.readlines()

In [32]:
composite_list = [lines[x:x+4] for x in range(0, len(lines),4)]
composite_list[0]

['<pos>\n',
 '<Comedy>\n',
 'foresee find dispatch \n',
 'A startship has just been lunched by a ravenous lioness whom she calls Lady Mama. She is accompanied by a mountain lion daughter who is a tiger lover, and a dusky young man. The dusky young man becomes very popular with Lady Mama when the Dneprado is purchased, and Lady Mama befriends him. Later on the dusky young man becomes very popular with Lady Mama when the Dneprado is sold, even though he already has a good relationship with the lady. The dusky young man helps Lady Mama to collect the dossiers of gold that Lady Mama is collecting from the gold mines which are just about to be burned up. He is a loan shark who has stolen the gold from a gold mine which low-lifes call "the burn pit". Cliff gives this loan shark a ring which he uses on himself which he uses to jump out of the burning mine. Lady Mama takes the ring and trying to get the Dneprado\'s gold, she takes it in the other direction while hiking. She gets it on a tree a

In [37]:
import re

def process_text(text):
    new_text = []
    new_text.append(re.sub(r"[<>]", "", text[0].strip()))
    new_text.append(re.sub(r"[<>]", "", text[1].strip()))
    new_text.append(text[2].strip())
    new_text.append(text[3].strip())
    return new_text

In [39]:
text = list(map(process_text, composite_list))

In [40]:
text[0]

['pos',
 'Comedy',
 'foresee find dispatch',
 'A startship has just been lunched by a ravenous lioness whom she calls Lady Mama. She is accompanied by a mountain lion daughter who is a tiger lover, and a dusky young man. The dusky young man becomes very popular with Lady Mama when the Dneprado is purchased, and Lady Mama befriends him. Later on the dusky young man becomes very popular with Lady Mama when the Dneprado is sold, even though he already has a good relationship with the lady. The dusky young man helps Lady Mama to collect the dossiers of gold that Lady Mama is collecting from the gold mines which are just about to be burned up. He is a loan shark who has stolen the gold from a gold mine which low-lifes call "the burn pit". Cliff gives this loan shark a ring which he uses on himself which he uses to jump out of the burning mine. Lady Mama takes the ring and trying to get the Dneprado\'s gold, she takes it in the other direction while hiking. She gets it on a tree and climbs d

In [82]:
sentiments = [t[0] for t in text]


In [83]:
genres = [t[1] for t in text]


## import

In [67]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tqdm import tqdm


## sentiment

In [43]:
!pip3 install flair
import flair
from flair.nn import Classifier
from flair.data import Sentence

/Users/kexinchen/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# load the model
tagger = Classifier.load('sentiment')
predicted_sentiments = []

for t in text:
     
    sentence = Sentence(t[3])

    tagger.predict(sentence)
    label = sentence.get_labels()[0].value
    predicted_sentiments.append("pos" if label == "POSITIVE" else "neg")
    

In [75]:
print("accuracy: ", accuracy_score(sentiments, predicted_sentiments))
print("precision: ", precision_score(sentiments, predicted_sentiments, average="macro"))
print("recall: ", recall_score(sentiments, predicted_sentiments, average="macro"))
print("f1_score: ", f1_score(sentiments, predicted_sentiments, average="macro"))

accuracy:  0.6160714285714286
precision:  0.6191489361702127
recall:  0.6160714285714286
f1_score:  0.613576185509107


## genre

In [58]:
from transformers import pipeline
from sklearn.preprocessing import normalize

classifier = pipeline(task='zero-shot-classification', model='valhalla/distilbart-mnli-12-6')


In [70]:
predicted_genres_result = []

for i in tqdm(range(len(text))):

    
    sequences = text[i][3]
    genre = text[i][1]
    
    candidate_labels = ["Comedy", "Romance", "Action", "Thriller", "Horror", "Crime", "Science", "Fantasy"]
    hypothesis_template = "This is a summary of movie plot. The genre of this is {}."
    prediction = classifier(sequences = sequences, candidate_labels = candidate_labels, hypothesis_template = hypothesis_template, multi_label=True)
    
    # index = prediction['labels'].index(genre)
    # score = prediction['scores'][index]
    # print(score)
    predicted_genres_result.append(prediction)
    
predicted_genres_result

100%|█████████████████████████████████████████| 112/112 [30:06<00:00, 16.13s/it]


[{'sequence': 'A startship has just been lunched by a ravenous lioness whom she calls Lady Mama. She is accompanied by a mountain lion daughter who is a tiger lover, and a dusky young man. The dusky young man becomes very popular with Lady Mama when the Dneprado is purchased, and Lady Mama befriends him. Later on the dusky young man becomes very popular with Lady Mama when the Dneprado is sold, even though he already has a good relationship with the lady. The dusky young man helps Lady Mama to collect the dossiers of gold that Lady Mama is collecting from the gold mines which are just about to be burned up. He is a loan shark who has stolen the gold from a gold mine which low-lifes call "the burn pit". Cliff gives this loan shark a ring which he uses on himself which he uses to jump out of the burning mine. Lady Mama takes the ring and trying to get the Dneprado\'s gold, she takes it in the other direction while hiking. She gets it on a tree and climbs down to get it. It is in the othe

In [79]:
predicted_genres = []
for i in range(len(predicted_genres_result)):
    genre = text[i][1]
    prediction = predicted_genres_result[i]
    index = prediction['labels'].index(genre)
    score = prediction['scores'][index]
    if score > 0.85:
        predicted_genres.append(genre)
    else:
        predicted_genres.append(prediction['labels'][0])
predicted_genres

['Comedy',
 'Romance',
 'Action',
 'Romance',
 'Horror',
 'Crime',
 'Romance',
 'Romance',
 'Crime',
 'Horror',
 'Action',
 'Thriller',
 'Horror',
 'Crime',
 'Crime',
 'Fantasy',
 'Crime',
 'Romance',
 'Action',
 'Thriller',
 'Horror',
 'Horror',
 'Romance',
 'Fantasy',
 'Romance',
 'Romance',
 'Action',
 'Thriller',
 'Horror',
 'Crime',
 'Thriller',
 'Comedy',
 'Horror',
 'Romance',
 'Action',
 'Thriller',
 'Science',
 'Crime',
 'Science',
 'Fantasy',
 'Science',
 'Romance',
 'Action',
 'Horror',
 'Horror',
 'Horror',
 'Horror',
 'Fantasy',
 'Fantasy',
 'Romance',
 'Action',
 'Thriller',
 'Romance',
 'Romance',
 'Fantasy',
 'Fantasy',
 'Romance',
 'Romance',
 'Action',
 'Thriller',
 'Horror',
 'Crime',
 'Science',
 'Fantasy',
 'Crime',
 'Romance',
 'Action',
 'Thriller',
 'Horror',
 'Crime',
 'Crime',
 'Fantasy',
 'Comedy',
 'Romance',
 'Action',
 'Fantasy',
 'Horror',
 'Crime',
 'Crime',
 'Fantasy',
 'Science',
 'Romance',
 'Action',
 'Thriller',
 'Action',
 'Science',
 'Science',
 '

In [81]:
print("accuracy: ", accuracy_score(genres, predicted_genres))
print("precision: ", precision_score(genres, predicted_genres, average="macro"))
print("recall: ", recall_score(genres, predicted_genres, average="macro"))
print("f1_score: ", f1_score(genres, predicted_genres, average="macro"))

accuracy:  0.6607142857142857
precision:  0.6729541098736145
recall:  0.6607142857142857
f1_score:  0.6319995666888456


In [62]:
sequences = 'A startship has just been lunched by a ravenous lioness whom she calls Lady Mama. She is accompanied by a mountain lion daughter who is a tiger lover, and a dusky young man. The dusky young man becomes very popular with Lady Mama when the Dneprado is purchased, and Lady Mama befriends him. Later on the dusky young man becomes very popular with Lady Mama when the Dneprado is sold, even though he already has a good relationship with the lady. The dusky young man helps Lady Mama to collect the dossiers of gold that Lady Mama is collecting from the gold mines which are just about to be burned up. He is a loan shark who has stolen the gold from a gold mine which low-lifes call "the burn pit". Cliff gives this loan shark a ring which he uses on himself which he uses to jump out of the burning mine. Lady Mama takes the ring and trying to get the Dneprado\'s gold, she takes it in the other direction while hiking. She gets it on a tree and climbs down to get it. It is in the other direction is another person. She picks him up and they both go on their way. The trail gets shorter until the girl is dead. The trail is now very narrow and is steep. A guy calls the girl "Karmapa." She says to her "Karmapa,Karmapa" in a very unprofitable voice and he goes on, "Everyone is talking to you from every direction. Karmapa hands it to everybody, now go on a straight route". The guy, who is the guy "Karmapa," tells her to get to town. She gets to town as she is in a state of shock. She is the girl who was. The girl who was. She had married the old lady. He was a rich man, who is gone and goes on to take up the old lady\'s place. The old lady is the new lady as well. So is the new lady. The old lady is the new one. Lady Mama gives her ring to her cousin, who gets it on the other side. She throws the ring in the camel\'s mouth. He is a smuggler and he\'s happy for the ring. It\'s in the little girl gets it on the camel. The camel says "Karmapa,Karmapa." Lady Mama gets it and gets her ring! She says "Karmapa,Karmapa." The camel says'
candidate_labels = ["Comedy", "Romance", "Action", "Thriller", "Horror", "Crime", "Science", "Fantasy"]
hypothesis_template = "This is a summary of movie plot. The genre of this is {}."
prediction = classifier(sequences = sequences, candidate_labels = candidate_labels, hypothesis_template = hypothesis_template, multi_label=True)



In [63]:
prediction

{'sequence': 'A startship has just been lunched by a ravenous lioness whom she calls Lady Mama. She is accompanied by a mountain lion daughter who is a tiger lover, and a dusky young man. The dusky young man becomes very popular with Lady Mama when the Dneprado is purchased, and Lady Mama befriends him. Later on the dusky young man becomes very popular with Lady Mama when the Dneprado is sold, even though he already has a good relationship with the lady. The dusky young man helps Lady Mama to collect the dossiers of gold that Lady Mama is collecting from the gold mines which are just about to be burned up. He is a loan shark who has stolen the gold from a gold mine which low-lifes call "the burn pit". Cliff gives this loan shark a ring which he uses on himself which he uses to jump out of the burning mine. Lady Mama takes the ring and trying to get the Dneprado\'s gold, she takes it in the other direction while hiking. She gets it on a tree and climbs down to get it. It is in the other